# Azure Cognitive Search Vector Search Code Sample with Azure OpenAI
This code demonstrates how to use Azure Cognitive Search by using the Push API to insert vectors into your search index.
## Prerequisites
To run the code, install the following packages. Please use the latest pre-release version `pip install azure-search-documents --pre`. This sample currently uses version `11.4.0b11`.

In [1]:
! pip3 install azure-search-documents==11.4.0b11
! pip3 install openai
! pip3 install python-dotenv
! pip3 install PyMuPDF
! pip3 install tiktoken
! pip3 install ratelimit
! pip3 install tenacity


## Import required libraries and environment variables

In [2]:
# Import required libraries  
import os  
import json  
import openai
import fitz
import tiktoken
import re
import hashlib
import time
from ratelimit import limits, sleep_and_retry
import dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient, SearchIndexingBufferedSender  
from azure.search.documents.indexes import SearchIndexClient  

from azure.search.documents.indexes.models import (  
    ExhaustiveKnnVectorSearchAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    VectorSearchVectorizer,
    VectorSearchVectorizerKind,
    AzureOpenAIParameters,
    AzureOpenAIVectorizer,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    VectorSearch,
    HnswVectorSearchAlgorithmConfiguration,
    ExhaustiveKnnVectorSearchAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings, 
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,
    HnswParameters,  
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchProfile,
    AzureOpenAIParameters,
    AzureOpenAIVectorizer,
)  
  
# Configure environment variables  
dotenv.load_dotenv(dotenv_path="../.env", override=True)
service_endpoint = str(os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT"))
print(service_endpoint)
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
key = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
openai.api_type = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION") 
model: str = "text-embedding-ada-002" 
credential = AzureKeyCredential(key)


https://plutolearning-demo-2.search.windows.net


In [3]:
# client = SearchClient(endpoint=service_endpoint,
#                       index_name="employease-index-2",
#                       credential=credential)
# results = client.search(search_text="exam 1")
# for result in results:
#     print("{}: {})".format(result["filepath"], result["url"]))


ResourceNotFoundError: () The index 'employease-index-2' for service 'plutolearning-demo-2' was not found.
Code: 
Message: The index 'employease-index-2' for service 'plutolearning-demo-2' was not found.

In [4]:
def chunk_text(text: str, limit: int, encoding: tiktoken.Encoding):
    """Split a string into parts of given size without breaking words.
    
    Args:
        text (str): Text to split.
        limit (int): Maximum number of tokens per part.
        encoding (tiktoken.Encoding): Encoding to use for tokenization.
        
    Returns:
        list[str]: List of text parts.
        
    """
    tokens = encoding.encode(text)
    parts = []
    text_parts = []
    current_part = []
    current_count = 0

    for token in tokens:
        current_part.append(token)
        current_count += 1

        if current_count >= limit:
            parts.append(current_part)
            current_part = []
            current_count = 0

    if current_part:
        parts.append(current_part)

    # Convert the tokenized parts back to text
    for part in parts:
        text = [
            encoding.decode_single_token_bytes(token).decode("utf-8", errors="replace")
            for token in part
        ]
        text_parts.append("".join(text))

    return text_parts


## Helper to read the file
Returns an entire string of the pdf

In [5]:
def get_pages(file):
    pages = []
    with fitz.open(file) as doc:
        for page in doc:
            pages.append(page.get_text())
    return pages

In [ ]:
textbook = get_pages("../data-cleaning/syllabus.pdf")

print(textbook[0])

 
  
McGee – MANA 4322.015 - Fall 2023 -- 1 
 
 
MANA 4322 Section 015: Strategic Management 
Fall 2023 
Instructor Information 
Instructor:  Jeffrey E. McGee 
Office: COBA 229 
Office Hours: By Appointment 
Office Telephone Number: (817) 272-3166 
Faculty Profile: http://www.uta.edu/profiles/jmcgee  
E-mail:  jmcgee@uta.edu 
Course Information 
Section Information: MANA 4322-015  
Time and Place of Class Meetings: TuTH 11:00 a.m. - 12:30 p.m. COBA154.  
Text: Essentials of Strategic Management: The Quest for Competitive Advantage, 7th Edition. 
ISBN10: 1264124333 | ISBN13: 9781264124336 ISBN10# 1260261549 and ISBN13# 97812660261547. (5th through 
7th Editions will also be acceptable). Capstone® Business Simulation (online registration and fee). Download the 
Team Membership Guide from the website after registration. Capstone Business Simulation Online registration at 
Capsim.com – ID# C145441. 
Description of Course Content  
Strategic Management, as a field of study, considers the ov

## Create the json file on each of the chunks

In [ ]:
def create_json(textbook_chunks):
    data = []
    for idx, chunk in enumerate(textbook_chunks):
        data.append({
            "id": str(idx),
            "title": f"Title for Chunk {idx}",  # Placeholder title
            "content": chunk,
            "category": "Category"  # Placeholder category
        })
    return data


In [ ]:
textbook_data = create_json(chunks)
print(textbook_data)
with open("sample_text_syllabus.json", "w") as json_file:
    json.dump(textbook_data, json_file, indent=4)


[{'id': '0', 'title': 'Title for Chunk 0', 'content': ' \n  \nMcGee – MANA 4322.015 - Fall 2023 -- 1 \n \n \nMANA 4322 Section 015: Strategic Management \nFall 2023 \nInstructor Information \nInstructor:  Jeffrey E. McGee \nOffice: COBA 229 \nOffice Hours: By Appointment \nOffice Telephone Number: (817) 272-3166 \nFaculty Profile: http://www.uta.edu/profiles/jmcgee  \nE-mail:  jmcgee@uta.edu \nCourse Information \nSection Information: MANA 4322-015  \nTime and Place of Class Meetings: TuTH 11:00 a.m. - 12:30 p.m. COBA154.  \nText: Essentials of Strategic Management: The Quest for Competitive Advantage, 7th Edition. \nISBN10: 1264124333 | ISBN13: 9781264124336 ISBN10# 1260261549 and ISBN13# 97812660261547. (5th through \n7th Editions will also be acceptable). Capstone® Business Simulation (online registration and fee). Download the \nTeam Membership Guide from the website after registration. Capstone Business Simulation Online registration at \nCapsim.com – ID# C145441. \nDescription of

## Get all the files paths in all_chapters_pdf
Chapter chunking

In [ ]:
# Generate Document Embeddings using OpenAI Ada 002


directory_path = '../data-cleaning/all_chapters_pdf/'
files = []

for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    if os.path.isfile(file_path) and file_path.endswith(".pdf"):
        files.append(file_path)




## Create embeddings
Read your data, generate OpenAI embeddings and export to a format to insert your Azure Cognitive Search index:

In [ ]:
# Function to generate embeddings for title and content fields, also used for query embeddings
@sleep_and_retry
@limits(calls=100, period=5)
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="plutolearning-embeddings")
    embeddings = response['data'][0]['embedding']
    return embeddings


## Load the JSON file and create the embeddings


In [ ]:
with open('./sample_text_syllabus.json', 'r', encoding='utf-8') as file:
    input_data = json.load(file)

for item in input_data:
    title = item['title']
    content = item['content']
    content_embeddings = generate_embeddings(content)
    item['vector'] = content_embeddings

# Output embeddings to docVectors.json file
with open("./docVectorsSyllabus.json", "w") as f:
    json.dump(input_data, f)

## Create your search index
Create your search index schema and vector search configuration:

In [6]:
# Create a search index
index_client = SearchIndexClient(endpoint=service_endpoint, credential=credential)

fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String,
                    filterable=True),
    SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_profile="myHnswProfile"),
]

# Configure the vector search configuration  
vector_search = VectorSearch(
    algorithms=[
        HnswVectorSearchAlgorithmConfiguration(
            name="myHnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric="cosine"
            )
        ),
        ExhaustiveKnnVectorSearchAlgorithmConfiguration(
            name="myExhaustiveKnn",
            kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
            parameters=ExhaustiveKnnParameters(
                metric="cosine"
            )
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm="myHnsw",
            vectorizer="myOpenAI"
        ),
        VectorSearchProfile(
            name="myExhaustiveKnnProfile",
            algorithm="myExhaustiveKnn",
            vectorizer="myOpenAI"
        )
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            name="myOpenAI",
            kind="azureOpenAI",
            azure_open_ai_parameters=AzureOpenAIParameters(
                resource_uri=os.getenv("AZURE_OPENAI_ENDPOINT"),
                deployment_id=model,
                api_key=os.getenv("AZURE_OPENAI_API_KEY")
            )
    )  
]  

)
semantic_settings = SemanticSettings(configurations=[])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)

print(result.name)


new-pluto-demo


## Insert text and embeddings into vector store
Add texts and metadata from the JSON data to the vector store:

In [7]:
# Upload some documents to the index
with open('./docVectors.json', 'r') as file:  
    documents = json.load(file)  
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)
print(f"Uploaded {len(documents)} documents") 

Uploaded 1097 documents


If you are indexing a very large number of documents, you can use the `SearchIndexingBufferedSender` which is an optimized way to automatically index the docs as it will handle the batching for you:

In [ ]:
# Upload some documents to the index  
with open('../output/docVectors.json', 'r') as file:  
    documents = json.load(file)  
  
# Use SearchIndexingBufferedSender to upload the documents in batches optimized for indexing  
with SearchIndexingBufferedSender(  
    endpoint=service_endpoint,  
    index_name=index_name,  
    credential=credential,  
) as batch_client:  
    # Add upload actions for all documents  
    batch_client.upload_documents(documents=documents)  
print(f"Uploaded {len(documents)} documents in total")  


Uploaded 108 documents in total


## Perform a vector similarity search

This example shows a pure vector search using the vectorizable text query, all you need to do is pass in text and your vectorizer will handle the query vectorization.

In [ ]:
# Pure Vector Search
query = "tools for software development"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizableTextQuery(text=query, k=3, fields="vector")
# Use the below query to pass in the raw vector query instead of the query vectorization
# vector_query = RawVectorQuery(vector=generate_embeddings(query), k=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


Title: Azure DevOps
Score: 0.82971567
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.81866795
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, suc

This example shows a pure vector search using a raw vector query, in this example, you are responsible for generating the query vector.

In [ ]:
# Pure Vector Search
query = "tools for software development"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
# Use the below query to pass in the raw vector query instead of the query vectorization
vector_query = RawVectorQuery(vector=generate_embeddings(query), k=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


Title: Azure DevOps
Score: 0.82971567
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.81866795
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, suc

This example shows a pure vector search to demonstrate OpenAI's text-embedding-ada-002 multilingual capabilities.

In [ ]:
# Pure Vector Search multi-lingual (e.g 'tools for software development' in Dutch)  
query = "tools voor softwareontwikkeling"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizableTextQuery(text=query, k=3, fields="contentVector")
# Use the below query to pass in the raw vector query instead of the query vectorization
# vector_query = RawVectorQuery(vector=generate_embeddings(query), k=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  



Title: Azure DevOps
Score: 0.80402017
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.7967943
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, such

## Perform an Exhaustive KNN exact nearest neighbor search

This example shows how you can exhaustively search your vector index regardless of what index you have, HNSW or ExhaustiveKNN. You can use this to calculate the ground-truth values.

In [ ]:
# Pure Vector Search
query = "tools for software development"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizableTextQuery(text=query, k=3, fields="contentVector", exhaustive=True)
# Use the below query to pass in the raw vector query instead of the query vectorization
# vector_query = RawVectorQuery(vector=generate_embeddings(query), k=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


Title: Azure DevOps
Score: 0.8297154
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.8186685
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, such 

## Perform a Cross-Field Vector Search

This example shows a cross-field vector search that allows you to query multiple vector fields at the same time. Note, ensure that the same embedding model was used for the vector fields you decide to query.

In [ ]:
# Pure Vector Search
query = "tools for software development"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
vector_query = VectorizableTextQuery(text=query, k=3, fields="titleVector, contentVector")
# Use the below query to pass in the raw vector query instead of the query vectorization
# vector_query = RawVectorQuery(vector=generate_embeddings(query), k=3, fields="titleVector, contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


Title: Azure DevOps
Score: 0.03333333507180214
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.032786883413791656
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports var

## Perform a Multi-Vector Search

This example shows a cross-field vector search that allows you to query multiple vector fields at the same time by passing in multiple query vectors. Note, in this case, you can pass in query vectors from two different embedding models to the corresponding vector fields in your index.

In [ ]:
# Multi-Vector Search
query = "tools for software development"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)  
vector_query_1 = VectorizableTextQuery(text=query, k=3, fields="titleVector") 
vector_query_2 = VectorizableTextQuery(text=query, k=3, fields="contentVector")  
# vector_query_1 = RawVectorQuery(vector=generate_embeddings(query), k=3, fields="titleVector") 
# vector_query_2 = RawVectorQuery(vector=generate_embeddings(query), k=3, fields="contentVector")  
  
results = search_client.search(  
    search_text=None,  
    vector_queries=[vector_query_1, vector_query_1],
    select=["title", "content", "category"],
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


Title: Azure DevOps
Score: 0.03333333507180214
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.032786883413791656
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports var

## Perform a Pure Vector Search with a filter
This example shows how to apply filters on your index. Note, that you can choose whether you want to use Pre-Filtering (default) or Post-Filtering.

In [ ]:
# Pure Vector Search
query = "tools for software development"  
  
search_client = SearchClient(service_endpoint, index_name, credential=credential)
  
vector_query = VectorizableTextQuery(text=query, k=3, fields="contentVector")
# Use the below query to pass in the raw vector query instead of the query vectorization
# vector_query = RawVectorQuery(vector=generate_embeddings(query), k=3, fields="contentVector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter="category eq 'Developer Tools'",
    select=["title", "content", "category"],
)
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


Title: Azure DevOps
Score: 0.8297154
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.8186685
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, such 

## Perform a Hybrid Search

In [ ]:
# Hybrid Search
query = "scalable storage solution"  
  
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
vector_query = VectorizableTextQuery(text=query, k=3, fields="contentVector")
# Use the below query to pass in the raw vector query instead of the query vectorization
# vector_query = RawVectorQuery(vector=generate_embeddings(query), k=3, fields="contentVector")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    top=3
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


Title: Azure Storage
Score: 0.03306011110544205
Content: Azure Storage is a scalable, durable, and highly available cloud storage service that supports a variety of data types, including blobs, files, queues, and tables. It provides a massively scalable object store for unstructured data. Storage supports data redundancy and geo-replication, ensuring high durability and availability. It offers a variety of data access and management options, including REST APIs, SDKs, and Azure Portal. You can secure your data using encryption at rest and in transit.
Category: Storage

Title: Azure Table Storage
Score: 0.032258063554763794
Content: Azure Table Storage is a fully managed, NoSQL datastore that enables you to store and query large amounts of structured, non-relational data. It provides features like automatic scaling, schema-less design, and a RESTful API. Table Storage supports various data types, such as strings, numbers, and booleans. You can use Azure Table Storage to store and manage

## Perform a Semantic Hybrid Search

In [ ]:
# Semantic Hybrid Search
query = "what is azure sarch?"

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector_query = VectorizableTextQuery(text=query, k=3, fields="contentVector")
# Use the below query to pass in the raw vector query instead of the query vectorization
# vector_query = RawVectorQuery(vector=generate_embeddings(query), k=3, fields="contentVector")

results = search_client.search(  
    search_text=query,  
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    query_type=QueryType.SEMANTIC, query_language=QueryLanguage.EN_US, semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")


Semantic Answer: Azure Cognitive Search is<em> a fully managed search-as-a-service that enables you to build rich search experiences for your applications.</em> It provides features like full-text search, faceted navigation, and filters. Azure Cognitive Search supports various data sources, such as Azure SQL Database, Azure Blob Storage, and Azure Cosmos DB.
Semantic Answer Score: 0.9462890625

Title: Azure Stack Edge
Reranker Score: 2.075716972351074
Content: Azure Stack Edge is a managed, edge computing appliance that enables you to run Azure services and AI workloads on-premises or at the edge. It provides features like hardware-accelerated machine learning, local caching, and integration with Azure IoT Hub. Azure Stack Edge supports various Azure services, such as Azure Functions, Azure Machine Learning, and Azure Kubernetes Service. You can use Azure Stack Edge to build edge computing applications, optimize your data processing, and ensure the security and compliance of your workl